In [1]:
import numpy as np
import pandas as pd
import pyspark
import sys

In [2]:
import pyspark.pandas as ps
import pyspark.sql.functions as fn
from pyspark.sql import SparkSession


In [3]:
# yarn mode
spark = SparkSession\
        .builder\
        .master("yarn")\
        .config('spark.executor.instances','999')\
        .config('spark.executor.memory','5500M')\
        .appName("thchow")\
        .getOrCreate()

22/10/19 09:58:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Check spark app name
spark.sparkContext.appName

'thchow'

In [5]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", True)

In [6]:
ps.set_option("compute.default_index_type", "distributed")

In [7]:
# print runtime versions
# Python version
sys.version

'3.10.6 (main, Aug 10 2022, 11:40:04) [GCC 11.3.0]'

In [8]:
# Spark version
spark.version

'3.3.0'

In [9]:
# load iris.csv into Spark dataframe
#df = spark.read.csv('file:///vagrant/data/iris.csv', header=True, inferSchema=True)
df = ps.read_csv('/tmp/grouped_simplified.csv')

/usr/local/lib/python3.10/dist-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [10]:
sdf=df.to_spark()

/usr/local/lib/python3.10/dist-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [11]:
sdf.createOrReplaceTempView("table1")
sdf2 = spark.sql("SELECT from_unixtime(open_time/1000, 'yyyy-MM-dd') as open_day, crypto, open, high, low, close, volume from table1")
sdf2.show()

+----------+------+-------+-------+-------+-------+--------+
|  open_day|crypto|   open|   high|    low|  close|  volume|
+----------+------+-------+-------+-------+-------+--------+
|2020-04-07|  DATA|0.04088|0.08999|0.04088|0.04741| 21344.9|
|2020-04-07|  DATA|0.04741|0.04741|0.04539|0.04539| 18031.6|
|2020-04-07|  DATA|0.04541|   0.06|0.04541|0.05154|360683.7|
|2020-04-07|  DATA|0.04698|0.05047|0.04636|0.04651|209030.1|
|2020-04-07|  DATA|0.04671|0.04671| 0.0461| 0.0461| 95900.6|
|2020-04-07|  DATA|0.04648|0.04701|0.04625|  0.047|105975.6|
|2020-04-07|  DATA| 0.0467|0.04701|0.04657|0.04678| 89238.7|
|2020-04-07|  DATA|0.04694|0.04737|0.04662|0.04737|113921.5|
|2020-04-07|  DATA|0.04737|0.04775|0.04737|0.04762| 33868.8|
|2020-04-07|  DATA|0.04782|0.04875|  0.047|  0.047|249964.1|
|2020-04-07|  DATA|0.04753|0.04753|0.04701|0.04701| 29641.0|
|2020-04-07|  DATA|0.04706| 0.0475|  0.047|  0.047|201356.0|
|2020-04-07|  DATA|0.04699|  0.047|0.04662|0.04662| 69740.6|
|2020-04-07|  DATA|0.046

In [12]:
sdf2.createOrReplaceTempView("table2")
sdf3 = spark.sql("SELECT open_day, crypto, round(first(open),4) as open, round(max(high),4) as high, round(min(low),4) as low, round(last(close),4) as close, round(sum(volume),4) as volume from table2 group by crypto, open_day")
sdf3.show()

+----------+------+------+------+------+------+------------+
|  open_day|crypto|  open|  high|   low| close|      volume|
+----------+------+------+------+------+------+------------+
|2020-05-23|  DATA|0.0712|0.0723|0.0704|0.0717|   3299715.8|
|2020-05-26|  DATA|0.0726|0.0729|0.0699|0.0701|   3579597.6|
|2020-06-13|  DATA|0.0567| 0.065|0.0561|0.0583|1.08368861E7|
|2020-09-02|  DATA| 0.059| 0.059|0.0532|0.0536|   3204210.0|
|2020-12-15|  DATA|0.0408|0.0424|0.0404|0.0407|   8683413.9|
|2021-12-26|  DATA|0.1236|0.1239|0.1195|0.1213|1.02392535E7|
|2022-02-07|  DATA| 0.092|0.0947|0.0842|0.0927|5.59320658E7|
|2022-02-13|  DATA|0.0828|0.0878|0.0826|0.0858|   9138100.6|
|2022-04-08|  DATA|0.0856|0.0876|0.0805|0.0837|7.89970345E7|
|2020-12-02|  COMP|108.95|110.86|104.11|110.18|  28394.8845|
|2021-01-25|  DATA|0.0624| 0.063|0.0583|0.0593|1.75427117E7|
|2021-01-27|  DATA|0.0555|0.0615|0.0508| 0.054| 1.5222135E7|
|2022-05-05|  DATA|0.0632|0.0672|0.0605|0.0607|1.94884531E7|
|2022-07-15|  DATA|0.031